In [0]:
import dlt
import pyspark.sql.functions as F


@dlt.table(
    name = "bronze_orders",
    table_properties = {
        "quality": "bronze"
    },
    comment = "Raw orders ingestion"
)
def create_bronze_orders():
    return (spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("cloudFiles.inferColumnTypes", "true")
        .load("/Volumes/circuitbox/landing/operational_data/orders/")
        .select(
            "*",
            F.col("_metadata.file_path").alias("input_file_path"),
            F.current_timestamp().alias("ingest_timstamp")
        ) 
    )  


---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-5178336860016380>, line 1
----> 1 import dlt;
      2 import pyspark.sql.functions as F
      5 @dlt.table(
      6     name = "bronze_orders",
      7     table_properties = {
   (...)
     11 )
     12 def create_bronze_orders():

File /databricks/python_shell/lib/dbruntime/autoreload/discoverability/autoreload_discoverability_hook.py:96, in AutoreloadDiscoverabilityHook._patched_import(self, name, *args, **kwargs)
     90 if not self._should_hint and (
     91     (module := sys.modules.get(absolute_name)) is not None and
     92     (fname := get_allowed_file_name_or_none(module)) is not None and
     93     (mtime := os.stat(fname).st_mtime) > self.last_mtime_by_modname.get(
     94         absolute_name, float("inf")) and not self._should_hint):
     95     self._should_hint = True
---> 96 module = self._original_bui

In [0]:
@dlt.table(
    name = "silver_orders_clean",
    table_properties = {
        "quality": "silver"
    },
    comment = "Cleaned orders",
)
@dlt.expect_or_fail("valid_customer_id", "customer_id IS NOT NULL")
@dlt.expect_or_drop("valid_order_id", "order_id IS NOT NULL")
@dlt.expect("valid_order_status", "order_status IN ('Pending','Shipped','Cancelled','Completed')")
@dlt.expect("valid_payment_method", "payment_method IN ('Credit Card','Bank Transfer','PayPal')")
def create_silver_orders_clean():
    return (spark.readStream.table("LIVE.bronze_orders").select(
            "order_id",
            "customer_id",
            F.col("order_timestamp").cast("timestamp").alias('order_timestamp'),
            "payment_method",
            "items",
            "order_status"
        )
    )

In [0]:
@dlt.table(
    name = "silver_orders",
    table_properties = {
        "quality": "silver"
    },
    comment = "sliver orders",
)

def create_silver_orders():
    return (spark.readStream.table("LIVE.silver_orders_clean")
            .withColumn("item", F.explode("items"))
            .select(
            "order_id",
            "customer_id",
            "order_timestamp",
            "payment_method",
            F.col("item.item_id").alias('item_id'),
            F.col("item.name").alias('name'),
            F.col("item.price").alias('price'),
            F.col("item.quantity").alias('quantity'),
            F.col("item.category").alias('category'),
            "order_status"
        )
    )